In [3]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

# Creating DataFrame from sample data
df = pd.read_csv('test_dataset_preferences - Sheet1.csv')

# Extract preferences for inputs and outputs from the last row
preferences = df.iloc[-1]
df = df.iloc[:-1]  # Remove the preferences row from the DMU data

# Defining the inputs and outputs for DEA
inputs = ['input_1', 'input_2']
outputs = ['output_1', 'output_2' , 'output_3']

# Store the efficiency scores and hypothetical units
efficiency_scores = []

# Iterating over each DMU and solving an LP for each one
for i, dmu in df.iterrows():
    # Create the linear programming problem for each DMU
    prob = LpProblem(f"DEA_{dmu['DMU']}", LpMaximize)

    # Define the variables for output weights and input weights
    output_weights = [LpVariable(f"u_{j}", lowBound=0) for j in range(len(outputs))]
    input_weights = [LpVariable(f"v_{i}", lowBound=0) for i in range(len(inputs))]

    # Objective function: maximize the weighted sum of outputs
    prob += lpSum([output_weights[j] * dmu[outputs[j]] for j in range(len(outputs))])

    # Add constraints for the other DMUs: weighted outputs ≤ weighted inputs for each DMU
    for k, other_dmu in df.iterrows():
        prob += lpSum([output_weights[j] * other_dmu[outputs[j]] for j in range(len(outputs))]) <= \
                lpSum([input_weights[i] * other_dmu[inputs[i]] for i in range(len(inputs))])

    # Constraint: weighted sum of inputs for the current DMU = 1
    prob += lpSum([input_weights[i] * dmu[inputs[i]] for i in range(len(inputs))]) == 1

    # Add weight preferences as constraints
    for j in range(len(outputs)):
        prob += output_weights[j] >= preferences[outputs[j]]  # Output weight preference constraint

    for i in range(len(inputs)):
        prob += input_weights[i] >= preferences[inputs[i]]  # Input weight preference constraint

    # Solve the linear program
    prob.solve()

    # Get the efficiency score for the current DMU
    efficiency_score = value(prob.objective)
    efficiency_scores.append(efficiency_score)

# Print Efficiency Scores
print("Efficiency Scores:")
for dmu, score in zip(df['DMU'], efficiency_scores):
    print(f"{dmu}: {score:.4f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/b798cbcac2734f3d889a7e2452b179b0-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/b798cbcac2734f3d889a7e2452b179b0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 46 RHS
At line 57 BOUNDS
At line 58 ENDATA
Problem MODEL has 10 rows, 5 columns and 27 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj -0 Primal inf 3.1256939 (6) Dual inf 10.006585 (3)
0  Obj -0 Primal inf 3.1256939 (6) Dual inf 3.6942037e+10 (5)
4  Obj 0.792 Primal inf 2.6927438 (5)
Primal infeasible - objective value 0

In [4]:
efficiency_scores

[0.792, 1.0000000348, 1.000000044, 5.85]

In [6]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

df = pd.read_csv('test_dataset_preferences - Sheet1.csv')

# Extract preferences for inputs and outputs from the last row
preferences = df.iloc[-1]
df = df.iloc[:-1]  # Remove the preferences row from the DMU data

# Defining the inputs and outputs for DEA
inputs = ['input_1', 'input_2']
outputs = ['output_1', 'output_2' , 'output_3']

# Store the efficiency scores
efficiency_scores = []

# Iterating over each DMU and solving an LP for each one
for i, dmu in df.iterrows():
    # Create the linear programming problem for each DMU
    prob = LpProblem(f"DEA_{dmu['DMU']}", LpMaximize)

    # Define the variables for output weights and input weights
    output_weights = [LpVariable(f"u_{j}", lowBound=0) for j in range(len(outputs))]
    input_weights = [LpVariable(f"v_{i}", lowBound=0) for i in range(len(inputs))]

    # Objective function: maximize the weighted sum of outputs (efficiency ratio)
    prob += lpSum([output_weights[j] * dmu[outputs[j]] for j in range(len(outputs))])

    # Add constraints for all DMUs to ensure weighted outputs <= weighted inputs (efficiency ≤ 1)
    for k, other_dmu in df.iterrows():
        prob += lpSum([output_weights[j] * other_dmu[outputs[j]] for j in range(len(outputs))]) <= \
                lpSum([input_weights[i] * other_dmu[inputs[i]] for i in range(len(inputs))])

    # Constraint: weighted sum of inputs for the current DMU = 1 (normalization)
    prob += lpSum([input_weights[i] * dmu[inputs[i]] for i in range(len(inputs))]) == 1

    # Add weight preferences as constraints
    for j in range(len(outputs)):
        prob += output_weights[j] >= preferences[outputs[j]]  # Output weight preference constraint

    for i in range(len(inputs)):
        prob += input_weights[i] >= preferences[inputs[i]]  # Input weight preference constraint

    # Solve the linear program
    prob.solve()

    # Get the efficiency score for the current DMU
    efficiency_score = value(prob.objective)
    
    # Ensure efficiency is no greater than 1
    if efficiency_score > 1:
        efficiency_score = 1.0

    efficiency_scores.append(efficiency_score)

# Print Efficiency Scores
print("Efficiency Scores:")
for dmu, score in zip(df['DMU'], efficiency_scores):
    print(f"{dmu}: {score:.4f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/18b1466d36f0423e821370d901e7ea75-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/18b1466d36f0423e821370d901e7ea75-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 46 RHS
At line 57 BOUNDS
At line 58 ENDATA
Problem MODEL has 10 rows, 5 columns and 27 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj -0 Primal inf 3.1256939 (6) Dual inf 10.006585 (3)
0  Obj -0 Primal inf 3.1256939 (6) Dual inf 3.6942037e+10 (5)
4  Obj 0.792 Primal inf 2.6927438 (5)
Primal infeasible - objective value 0

In [7]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value


# Creating DataFrame from sample data
df = pd.read_csv('test_dataset_preferences - Sheet1.csv')

# Extract preferences for inputs and outputs from the last row
preferences = df.iloc[-1]
df = df.iloc[:-1]  # Remove the preferences row from the DMU data

# Defining the inputs and outputs for DEA
inputs = ['input_1', 'input_2']
outputs = ['output_1', 'output_2' , 'output_3']

# Store the efficiency scores and hypothetical units
efficiency_scores = []
hypothetical_inputs = []
hypothetical_outputs = []

# Iterating over each DMU and solving an LP for each one
for i, dmu in df.iterrows():
    # Create the linear programming problem for each DMU
    prob = LpProblem(f"DEA_{dmu['DMU']}", LpMaximize)

    # Define the variables for output weights and input weights
    output_weights = [LpVariable(f"u_{j}", lowBound=0) for j in range(len(outputs))]
    input_weights = [LpVariable(f"v_{i}", lowBound=0) for i in range(len(inputs))]

    # Objective function: maximize the weighted sum of outputs (efficiency ratio)
    prob += lpSum([output_weights[j] * dmu[outputs[j]] for j in range(len(outputs))])

    # Add constraints for all DMUs to ensure weighted outputs ≤ weighted inputs (efficiency ≤ 1)
    for k, other_dmu in df.iterrows():
        prob += lpSum([output_weights[j] * other_dmu[outputs[j]] for j in range(len(outputs))]) <= \
                lpSum([input_weights[i] * other_dmu[inputs[i]] for i in range(len(inputs))])

    # Constraint: weighted sum of inputs for the current DMU = 1 (normalization)
    prob += lpSum([input_weights[i] * dmu[inputs[i]] for i in range(len(inputs))]) == 1

    # Add weight preferences as constraints
    for j in range(len(outputs)):
        prob += output_weights[j] >= preferences[outputs[j]]  # Output weight preference constraint

    for i in range(len(inputs)):
        prob += input_weights[i] >= preferences[inputs[i]]  # Input weight preference constraint

    # Solve the linear program
    prob.solve()

    # Get the efficiency score for the current DMU
    efficiency_score = value(prob.objective)
    
    # Ensure efficiency is no greater than 1
    if efficiency_score > 1:
        efficiency_score = 1.0

    efficiency_scores.append(efficiency_score)

    # If the DMU is inefficient (score < 1), calculate the hypothetical composite unit
    if efficiency_score < 1:
        # Lambda variables represent the weights of the composite unit for each DMU
        lambda_vars = [LpVariable(f"lambda_{k}", lowBound=0) for k in range(len(df))]
        
        # Create another LP problem to find the composite unit
        prob_composite = LpProblem(f"Composite_{dmu['DMU']}", LpMaximize)
        
        # Composite unit inputs (weighted sum of inputs from other DMUs)
        composite_inputs = [lpSum([lambda_vars[k] * df.iloc[k][inputs[j]] for k in range(len(df))]) for j in range(len(inputs))]
        
        # Composite unit outputs (weighted sum of outputs from other DMUs)
        composite_outputs = [lpSum([lambda_vars[k] * df.iloc[k][outputs[j]] for k in range(len(df))]) for j in range(len(outputs))]
        
        # Objective function: maximize sum of lambda variables (this ensures we are creating a composite unit)
        prob_composite += lpSum(lambda_vars)
        
        # Constraints: composite unit inputs and outputs should match the inefficient DMU’s efficiency-adjusted input/output levels
        for j in range(len(inputs)):
            prob_composite += composite_inputs[j] == dmu[inputs[j]] * efficiency_score
        
        for j in range(len(outputs)):
            prob_composite += composite_outputs[j] >= dmu[outputs[j]]
        
        # Solve the composite unit problem
        prob_composite.solve()

        # Extract the composite inputs and outputs (hypothetical values)
        hypothetical_input_values = [value(composite_inputs[j]) for j in range(len(inputs))]
        hypothetical_output_values = [value(composite_outputs[j]) for j in range(len(outputs))]

        hypothetical_inputs.append(hypothetical_input_values)
        hypothetical_outputs.append(hypothetical_output_values)
    else:
        # If the DMU is efficient, use the original input/output values
        hypothetical_inputs.append(list(dmu[inputs]))
        hypothetical_outputs.append(list(dmu[outputs]))

# Print Efficiency Scores
print("Efficiency Scores:")
for dmu, score in zip(df['DMU'], efficiency_scores):
    print(f"{dmu}: {score:.4f}")

# Print Hypothetical Composite Inputs and Outputs for Inefficient DMUs
print("\nHypothetical Composite Unit Input and Output Values for Inefficient DMUs:")
for i, (dmu, score) in enumerate(zip(df['DMU'], efficiency_scores)):
    if score < 1:
        print(f"\n{dmu} (Inefficient DMU):")
        print(f"Hypothetical Composite Inputs: {hypothetical_inputs[i]}")
        print(f"Hypothetical Composite Outputs: {hypothetical_outputs[i]}")
    else:
        print(f"\n{dmu} (Efficient DMU): No adjustments needed.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/69f24caf3ea6461e9f909b2c8bba3010-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/69f24caf3ea6461e9f909b2c8bba3010-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 46 RHS
At line 57 BOUNDS
At line 58 ENDATA
Problem MODEL has 10 rows, 5 columns and 27 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj -0 Primal inf 3.1256939 (6) Dual inf 10.006585 (3)
0  Obj -0 Primal inf 3.1256939 (6) Dual inf 3.6942037e+10 (5)
4  Obj 0.792 Primal inf 2.6927438 (5)
Primal infeasible - objective value 0